In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import rebound
import matplotlib
matplotlib.use('Agg')
from animate import *


In [2]:
sim = rebound.Simulation()
sim.units = ('AU', 'Yr', 'Msun')
labels = ["Sun", "Neptune", "Pluto"]
sim.add(labels)

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')
Searching NASA Horizons for 'Pluto'... 
Found: Pluto Barycenter (9) (chosen from query 'Pluto')


In [3]:
os = sim.calculate_orbits()
sim.integrator = "ias15"
Nout = 1000            # number of points to display
tmax = 10*os[0].P     # let the simulation run for 10 neptune orbits

In [4]:
x = np.zeros((3,Nout))
y = np.zeros((3,Nout))
trails = []
times = np.linspace(0.,tmax,Nout)

In [5]:
ps=sim.particles
sim.move_to_com()
for i,time in enumerate(times):
    sim.integrate(time)
    for j in range(3):
        x[j][i] = ps[j].x 
        y[j][i] = ps[j].y
    trails.append([np.array(ps[j+1].sample_orbit()) for j in range(2)])

In [6]:
dr = np.sqrt((x[2]-x[1])**2 + (y[2]-y[1])**2)

In [7]:
path = './plot_NP_rotshort/'

In [8]:
rot = -np.arctan2(y[1],x[1]) #subtract off rotation of Neptune

In [9]:
def rotmatrix(x,y,theta):
    return x*np.cos(theta) - y*np.sin(theta), x*np.sin(theta) + y*np.cos(theta)

In [10]:
def makeplot(i):
    fig, ax = plt.subplots(ncols=2,figsize=(15,7))
    ps = sim.particles

    # manually set plot boundaries
    lim = 50
    ax[0].set_xlim([-lim, lim])
    ax[0].set_ylim([-lim, lim])
    # plot the stars and planets with separate symbols
    x0,y0 = rotmatrix(x[0],y[0],rot)
    x1,y1 = rotmatrix(x[1],y[1],rot)
    x2,y2 = rotmatrix(x[2],y[2],rot)
    ax[0].scatter(x0[i],y0[i], s=50, marker='*', facecolor='black', zorder=3)
    ax[0].scatter(x1[i],y1[i], s=20, facecolor='black', zorder=3)
    ax[0].scatter(x2[i],y2[i], s=20, facecolor='black', zorder=3)
    ax[0].plot(x2[:i],y2[:i], c='red', zorder=3,alpha=0.5)

    trailx1, traily1 = rotmatrix(trails[i][0][:,0], trails[i][0][:,1], rot[i])
    trailx2, traily2 = rotmatrix(trails[i][1][:,0], trails[i][1][:,1], rot[i])

#    ax[0].add_collection(fading_line(trailx1,traily1))
#    ax[0].add_collection(fading_line(trailx2,traily2))
    ax[0].set_xlabel('x [AU]')
    ax[0].set_ylabel('y [AU]')


    ax[1].plot(times[:i],dr[:i])
    ax[1].scatter(times,dr,c='w',alpha=0)
    ax[1].set_xlabel('Time [yrs]')
    ax[1].set_ylabel('Distance between Nep and Pluto [AU]')
    ax[1].set_ylim(ymin=0)
    plt.savefig(path+str(i)+'.png',dpi=300,bbox_inches='tight')
    
    return fig

In [ ]:
savefigures(makeplot,np.arange(0,Nout,5),path)

saving figures


 32%|█████████████▏                            | 63/200 [00:47<01:42,  1.33it/s]

In [ ]:
render(path,'NP_rotshort','gifbyavi',runtime=10,cleanup_type='rm')